데이터 전처리 없이 머신러닝 돌리기

In [1]:
# 라이브러리
import os
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import torch
import random

def reset_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)    # 파이썬 환경변수 시드 고정
    np.random.seed(seed)
    torch.manual_seed(seed) # cpu 연산 무작위 고정
    torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정
    torch.backends.cudnn.deterministic = True  # cuda 라이브러리에서 Deterministic(결정론적)으로 예측하기 (예측에 대한 불확실성 제거 )

In [2]:
# hotel폴더의 archive폴더에 있는 CSV로드
import easydict
args = easydict.EasyDict()

args.default_path = './archive/hotel_bookings.csv'

# 데이터 분석을 위한 변수들
args.random_state = 21

데이터 로드

In [3]:
plt.style.use('fivethirtyeight')
plt.ion()

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(args.default_path)
df.shape

(119390, 32)

In [5]:
df["country"].mode()[0]

'PRT'

In [6]:
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [7]:
# company, agent, children → 0으로 채우기
df["company"] = df["company"].fillna(0)
df["agent"] = df["agent"].fillna(0)
df["children"] = df["children"].fillna(0)

# country → 최빈값으로 채우기
df["country"] = df["country"].fillna(df["country"].mode()[0])

train, test 분리

In [8]:
from sklearn.model_selection import train_test_split

# 전체 train/test 먼저
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['is_canceled'])

# train 안에서 다시 eval 분리
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['is_canceled'])

print("Train:", train_df.shape)
print("Eval :", eval_df.shape)
print("Test :", test_df.shape)
# 과적합을 막기 위해 train, eval, test로 나눔

Train: (76409, 32)
Eval : (19103, 32)
Test : (23878, 32)


feature 생성

In [9]:
from sklearn.ensemble import StackingClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [10]:
# X, y 정의
X = df.drop(["is_canceled", "reservation_status", "reservation_status_date", "deposit_type", "agent", "assigned_room_type"], axis=1)
y = df["is_canceled"]

# train/test split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

feature 생성

In [11]:
# Create 'total_guests' on both the training and testing sets
X_tr['total_guests'] = X_tr['adults'] + X_tr['children'] + X_tr['babies']
X_te['total_guests'] = X_te['adults'] + X_te['children'] + X_te['babies']

# Create 'is_alone' feature for both sets
# 1 if total_guests is 1, otherwise 0
X_tr['is_alone'] = X_tr['total_guests'].apply(lambda x: 1 if x == 1 else 0)
X_te['is_alone'] = X_te['total_guests'].apply(lambda x: 1 if x == 1 else 0)

# Verify the new feature has been added
print("X_tr shape:", X_tr.shape)
print("X_te shape:", X_te.shape)

# Optionally, you can drop the intermediate 'total_guests' feature
X_tr = X_tr.drop('total_guests', axis=1)
X_te = X_te.drop('total_guests', axis=1)

X_tr shape: (95512, 28)
X_te shape: (23878, 28)


company, agent  
회사와 대행사가 미치는 영향 분석

In [12]:
# X_te[company]의 값이 0보다 크면1 / 작으면 0# company 값이 0보다 크면 1, 아니면 0  
# .astype(int) : Boolean을 정수로 변환
X_te['has_company'] = (X_te['company'] > 0).astype(int)
X_tr['has_company'] = (X_tr['company'] > 0).astype(int)
# 확인
print("has_company 분포:")
print(X_tr['has_company'].value_counts()/len(X_tr['has_company']))

has_company 분포:
has_company
0    0.943452
1    0.056548
Name: count, dtype: float64


In [13]:
# X_te[company]의 값이 0보다 크면1 / 작으면 0# company 값이 0보다 크면 1, 아니면 0  
# .astype(int) : Boolean을 정수로 변환
X_te['has_company'] = (X_te['company'] > 0).astype(int)
X_tr['has_company'] = (X_tr['company'] > 0).astype(int) 
# 확인
print("has_company 분포:")
print(X_tr['has_company'].value_counts()/len(X_tr['has_company']))

has_company 분포:
has_company
0    0.943452
1    0.056548
Name: count, dtype: float64


meal 형태  
FB 즉 식사를 3번 제공하면 취소율이 높았음

In [14]:
# meal이 FB이면 1, 아니면 0인 피처생성
import numpy as np

# np.where로 조건부 값 할당
X_tr['is_FB_meal'] = np.where(X_tr['meal'] == 'FB', 1, 0)
X_te['is_FB_meal'] = np.where(X_te['meal'] == 'FB', 1, 0)

# 확인
print("is_FB_meal 분포:")
print(X_tr['is_FB_meal'].value_counts()/len(X_tr['is_FB_meal']))

is_FB_meal 분포:
is_FB_meal
0    0.993362
1    0.006638
Name: count, dtype: float64


예약한 방과 실제 방이 다른경우 끼치는 영향

In [15]:
import numpy as np

# 'is_room_changed' 컬럼 생성
df['is_room_changed'] = np.where(df['reserved_room_type'] == df['assigned_room_type'], 0, 1)

# 새로 생성된 피처 확인
print(df[['reserved_room_type', 'assigned_room_type', 'is_room_changed']].head())

  reserved_room_type assigned_room_type  is_room_changed
0                  C                  C                0
1                  C                  C                0
2                  A                  C                1
3                  A                  A                0
4                  A                  A                0


ard 이상치 제거  
1~200 사이의 값을 제외하고 전부 중앙값으로 처리

In [16]:
import numpy as np

# 1. IQR을 사용하여 훈련 데이터(X_tr)의 이상치 범위 계산
Q1 = X_tr['adr'].quantile(0.25)
Q3 = X_tr['adr'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# 2. 이상치(outliers)를 제외한 훈련 데이터의 adr 중앙값 재계산
# adr은 보통 0보다 크므로 하한선을 0으로 설정하거나 IQR로 계산된 값을 사용
# 여기서는 IQR로 계산된 값을 사용하여 더 일반적인 방법으로 처리
adr_filtered_median = X_tr.loc[(X_tr['adr'] >= lower_bound) & (X_tr['adr'] <= upper_bound), 'adr'].median()

# 3. 훈련 세트(X_tr)에 새로운 'adr_processed' 피처 생성
# 이상치 범위(lower_bound ~ upper_bound)를 벗어나는 값들을 필터링된 중앙값으로 대체
X_tr['adr_processed'] = np.where(
    (X_tr['adr'] < lower_bound) | (X_tr['adr'] > upper_bound),
    adr_filtered_median,
    X_tr['adr']
)

# 4. 테스트 세트(X_te)에 새로운 'adr_processed' 피처 생성
# 훈련 데이터에서 계산한 동일한 이상치 범위와 중앙값을 사용
X_te['adr_processed'] = np.where(
    (X_te['adr'] < lower_bound) | (X_te['adr'] > upper_bound),
    adr_filtered_median,
    X_te['adr']
)

# 처리된 피처의 통계량 확인
print("처리된 adr의 통계량 (X_tr):")
print(X_tr['adr_processed'].describe())

처리된 adr의 통계량 (X_tr):
count    95512.000000
mean        96.873297
std         39.938106
min         -6.380000
25%         69.290000
50%         92.000000
75%        120.000000
max        211.030000
Name: adr_processed, dtype: float64


숙박일 feature 생성

In [17]:
# X_tr에 'total_stay' 피처 생성
X_tr['total_stay'] = X_tr['stays_in_weekend_nights'] + X_tr['stays_in_week_nights']

# X_te에 'total_stay' 피처 생성
X_te['total_stay'] = X_te['stays_in_weekend_nights'] + X_te['stays_in_week_nights']

lead_time 이상치 제거  
IQR계산법을 통해 이상치를 제거한다.  
375보다 크면 중앙값을 넣음

In [18]:
# Calculate the median lead time from the TRAINING set
lead_time_median = X_tr['lead_time'].median()

# Create 'lead_time_processed' feature for the TRAINING set
# Values outside the 0 to 373 range are replaced with the training set median
X_tr['lead_time_processed'] = np.where(
    (X_tr['lead_time'] < 0) | (X_tr['lead_time'] > 373),
    lead_time_median,
    X_tr['lead_time']
)

# Create 'lead_time_processed' feature for the TESTING set
# Use the SAME median calculated from the training set
X_te['lead_time_processed'] = np.where(
    (X_te['lead_time'] < 0) | (X_te['lead_time'] > 373),
    lead_time_median,
    X_te['lead_time']
)


hotel 타입

In [19]:
# 'hotel' 컬럼을 기반으로 'is_resort' 피처 생성
# City Hotel은 0, Resort Hotel은 1로 변환
X_tr['is_resort'] = X_tr['hotel'].map({'City Hotel': 0, 'Resort Hotel': 1})
X_te['is_resort'] = X_te['hotel'].map({'City Hotel': 0, 'Resort Hotel': 1})


In [20]:
# X_tr과 X_te에 남아있는 object 타입(문자열) 컬럼들을 찾습니다.
cat_cols = X_tr.select_dtypes(include='object').columns.tolist()

if len(cat_cols) > 0:
    print("다음과 같은 문자열 컬럼이 남아있습니다:", cat_cols)
    
    # X_tr과 X_te에 남아있는 문자열 컬럼들을 원-핫 인코딩합니다.
    X_tr = pd.get_dummies(X_tr, columns=cat_cols, drop_first=True)
    X_te = pd.get_dummies(X_te, columns=cat_cols, drop_first=True)
    
    # 훈련 세트와 테스트 세트의 컬럼을 일치시킵니다.
    X_te = X_te.reindex(columns=X_tr.columns, fill_value=0)
else:
    print("모든 문자열 컬럼이 숫자형으로 변환되었습니다.")

다음과 같은 문자열 컬럼이 남아있습니다: ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'customer_type']


In [21]:
# 남아있는 column 및 feature 확인
print("X_tr columns:", X_tr.columns)

X_tr columns: Index(['lead_time', 'arrival_date_year', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest',
       ...
       'reserved_room_type_D', 'reserved_room_type_E', 'reserved_room_type_F',
       'reserved_room_type_G', 'reserved_room_type_H', 'reserved_room_type_L',
       'reserved_room_type_P', 'customer_type_Group',
       'customer_type_Transient', 'customer_type_Transient-Party'],
      dtype='object', length=235)


In [22]:
# 제거할 원본 컬럼 리스트
columns_to_drop = [
    'hotel',
    'lead_time',
    'adr',
    'stays_in_weekend_nights',
    'stays_in_week_nights',
    'total_guests',
    'reserved_room_type',
    'assigned_room_type',
    'customer_type',
]

# X_tr에서 컬럼 제거
X_tr = X_tr.drop(columns=columns_to_drop, errors='ignore')

# X_te에서 컬럼 제거
X_te = X_te.drop(columns=columns_to_drop, errors='ignore')

# 제거 후 남은 컬럼 확인
print("X_tr의 최종 컬럼 목록:")
print(X_tr.columns)

print("\nX_te의 최종 컬럼 목록:")
print(X_te.columns)

X_tr의 최종 컬럼 목록:
Index(['arrival_date_year', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       ...
       'reserved_room_type_D', 'reserved_room_type_E', 'reserved_room_type_F',
       'reserved_room_type_G', 'reserved_room_type_H', 'reserved_room_type_L',
       'reserved_room_type_P', 'customer_type_Group',
       'customer_type_Transient', 'customer_type_Transient-Party'],
      dtype='object', length=231)

X_te의 최종 컬럼 목록:
Index(['arrival_date_year', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       ...
       'reserved_room_type_D', 'reserved_room_type_E', 'reserved_room_type_F',
       'reserved_room_type_G', 'reserved_room_type_H', 'reserved_room_type_L

xgBOOST

In [26]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# XGBoost 모델 인스턴스 생성
# `use_label_encoder=False`와 `eval_metric`은 경고를 피하기 위해 추가
xgb_classifier = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# 모델 직접 학습
xgb_classifier.fit(X_tr, y_tr)

# 예측 및 성능 지표 계산
y_tr_pred = xgb_classifier.predict(X_tr)
y_te_pred = xgb_classifier.predict(X_te)
y_tr_proba = xgb_classifier.predict_proba(X_tr)[:, 1]
y_te_proba = xgb_classifier.predict_proba(X_te)[:, 1]

# 훈련 성능 출력
print("XGBoost 훈련 성능:")
print(f"  정확도 (Accuracy): {accuracy_score(y_tr, y_tr_pred):.4f}")
print(f"  정밀도 (Precision): {precision_score(y_tr, y_tr_pred):.4f}")
print(f"  재현율 (Recall): {recall_score(y_tr, y_tr_pred):.4f}")
print(f"  F1-점수 (F1-Score): {f1_score(y_tr, y_tr_pred):.4f}")
print(f"  AUC: {roc_auc_score(y_tr, y_tr_proba):.4f}")

# 테스트 성능 출력
print("\nXGBoost 테스트 성능:")
print(f"  정확도 (Accuracy): {accuracy_score(y_te, y_te_pred):.4f}")
print(f"  정밀도 (Precision): {precision_score(y_te, y_te_pred):.4f}")
print(f"  재현율 (Recall): {recall_score(y_te, y_te_pred):.4f}")
print(f"  F1-점수 (F1-Score): {f1_score(y_te, y_te_pred):.4f}")
print(f"  AUC: {roc_auc_score(y_te, y_te_proba):.4f}")

XGBoost 훈련 성능:
  정확도 (Accuracy): 0.8787
  정밀도 (Precision): 0.8585
  재현율 (Recall): 0.8051
  F1-점수 (F1-Score): 0.8310
  AUC: 0.9531

XGBoost 테스트 성능:
  정확도 (Accuracy): 0.8664
  정밀도 (Precision): 0.8457
  재현율 (Recall): 0.7819
  F1-점수 (F1-Score): 0.8125
  AUC: 0.9417


## 모듈화된 파이프라인 실행
아래 셀들은 `hotel_mod` 패키지로 전처리와 학습 과정을 간결하게 실행합니다.


In [ ]:
# 모듈 임포트
from hotel_mod import settings, preprocess, modeling, utils

# 시드 고정
settings.reset_seeds(settings.RANDOM_STATE)

# 데이터 로드 및 파이프라인 전처리
import pandas as pd

df = pd.read_csv(settings.DEFAULT_PATH)
split = preprocess.pipeline(df, random_state=settings.RANDOM_STATE)

# 모델 학습 (XGBoost)
model = utils.train_xgb_classifier(split.X_train, split.y_train, random_state=settings.RANDOM_STATE)

# 예측 및 지표 계산
import numpy as np

y_tr_pred = model.predict(split.X_train)
y_te_pred = model.predict(split.X_test)
y_tr_proba = model.predict_proba(split.X_train)[:, 1]
y_te_proba = model.predict_proba(split.X_test)[:, 1]

m_tr = modeling.evaluate_binary(split.y_train, y_tr_pred, y_tr_proba)
m_te = modeling.evaluate_binary(split.y_test, y_te_pred, y_te_proba)

print(modeling.format_metrics("XGBoost 훈련 성능:", m_tr))
print()
print(modeling.format_metrics("XGBoost 테스트 성능:", m_te))
